# Structure checking tutorial

A complete checking analysis of a single structure follows.
use .revert_changes() at any time to recover the original structure

Structure checking is a key step before setting up a protein system for simulations. 
A number of normal issues found in structures at Protein Data Bank may compromise the success of the simulation, or may suggest that longer equilibration procedures are necessary.

The biobb_structure_checking modules allow to 
- Do basic manipulations on structures (selection of models, chains, alternative locations
- Detect and fix amide assignments, wrong chirality
- Detect and fix protein backbone issues (missing fragments, and atoms, capping)
- Detect and fix missing side-chain atoms
- Add hydrogen atoms according to several criteria
- Detect and classify clashes
- Detect possible SS bonds

biobb_structure_checking modules can used at the command line biobb_structure_checking/bin/check_structure


In [1]:
%load_ext autoreload
%autoreload 2

## Installation

#### Basic imports and initialization

In [2]:
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]
args = cts.set_defaults(base_dir_path,{'notebook':True})


## General help

In [3]:
with open(args['commands_help_path']) as help_file:
    print(help_file.read())


BioBB's check_structure.py performs MDWeb structure checking set as a command line
utility.

commands:     Help on available commands
command_list: Run all tests from conf file or command line list
checkall:     Perform all checks without fixes
load:         Stores structure on local cache and provides basic statistics

1. System Configuration
sequences [--output_fasta FASTA]
    Print canonical and structure sequences in FASTA format
    --output_fasta Write sequences found in an externla FASTA file
models [--select model_num] [--superimpose] [--save_split]
    Detect/Select Models
    --superimpose Superimposes currently selected models
    --save_split Split models as separated output files.
    --build_complex Build an actual complex from Biounits stored as collection of models
chains [--select chain_ids | molecule_type] [--rename (auto|new label)]
    [--renumber (auto | [A:]ini0[-fin0]=[B:]ini1)] [--rem_inscodes]
    Detect/Select Chains
    --select chains ids or chain types (P

Set input (PDB or local file, pdb or mmCif formats allowed) and output (local file, pdb format).  
Use pdb:pdbid for downloading structure from PDB (RCSB)

In [32]:
base_path = '/home/gelpi/DEVEL/BioPhysics/wdir/'
args['input_structure_path'] = base_path + '6m0j.cif'
args['output_structure_path'] = base_path + '6m0j_fixed.pdb'
args['output_structure_path_charges'] = base_path + '6m0j_fixed.pdbqt'
args['debug'] = False
args['verbose'] = False
#Added missing defaults
args['time_limit'] = 3600
args['nocache'] = False
args['copy_input'] = None
args['build_warnings'] = False
args['coords_only'] = False
args['overwrite'] = False
args['output_format'] = 'pdb'

Initializing checking engine, loading structure and showing statistics

In [33]:

st_c = StructureChecking(base_dir_path, args)


Canonical sequence for model 0:
Structure /home/gelpi/DEVEL/BioPhysics/wdir/6m0j.cif loaded
 PDB id: 6M0J 
 Title: Crystal structure of 2019-nCoV spike receptor-binding domain bound with ACE2
 Experimental method: X-RAY DIFFRACTION
 Keywords: VIRAL PROTEIN/HYDROLASE
 Resolution (A): 2.4500

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  876
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  85
 Num. ligands or modified residues:  5
 Num. water mol.:  80
 Num. atoms:  6543
Metal/Ion residues found
 ZN A901
Small mol ligands found
NAG A902
NAG A903
NAG A904
NAG E601


#### models
Checks for the presence of models in the structure. 
MD simulations require a single structure, although some structures (e.g. biounits) may be defined as a series of models, in such case all of them are usually required.  
Use models('--select N') to select model num N for further analysis

In [34]:
st_c.models()

Running models.
Detected 1 Model(s)
Found Single model


#### chains
Checks for chains (also obtained from print_stats), and allow to select one or more.   
MD simulations are usually performed with complete structures. However input structure may contain several copies of the system, or contains additional chains like peptides or nucleic acids that may be removed. 
Use chains('X,Y') to select chain(s) X and Y to proceed

In [35]:
st_c.chains()


Running chains.
Detected 2 Chain(s)
 A: Protein
 E: Protein


#### altloc
Checks for the presence of residues with alternative locations. Atoms with alternative coordinates and their occupancy are reported.  
MD simulations requires a single position for each atom.  
Use altloc('occupancy | alt_ids | list of res:id) to select the alternative


In [36]:
st_c.altloc()

Running altloc.
Detected 2 residues with alternative location labels
HIS A228
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  ND1  A (0.50) B (0.50)
  CD2  A (0.50) B (0.50)
  CE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)
GLN E493
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  CD   A (0.50) B (0.50)
  OE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)


We need to choose one of the alternative forms for each residue

In [37]:
st_c.altloc('occupancy')

Running altloc. Options: occupancy
Detected 2 residues with alternative location labels
HIS A228
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  ND1  A (0.50) B (0.50)
  CD2  A (0.50) B (0.50)
  CE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)
GLN E493
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  CD   A (0.50) B (0.50)
  OE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)
Selecting location occupancy


In [38]:
st_c.altloc()

Running altloc.
Detected no residues with alternative location labels


#### metals
Detects HETATM being metal ions allow to selectively remove them.  
To remove use metals (' All | None | metal_type list | residue list ')

In [39]:
st_c.metals()

Running metals.
Found 1 Metal ions
  ZN A901.ZN 


#### ligands
Detects HETATM (excluding Water molecules) to selectively remove them.  
To remove use ligands('All | None | Residue List (by id, by num)')


In [40]:
st_c.ligands()

Running ligands.
Detected 5 Ligands
  ZN A901
 NAG A902
 NAG A903
 NAG A904
 NAG E601


In [41]:
st_c.ligands('All')

Running ligands. Options: All
Detected 5 Ligands
  ZN A901
 NAG A902
 NAG A903
 NAG A904
 NAG E601
Canonical sequence for model 0:
Ligands removed All (5)


In [42]:
st_c.ligands()

Running ligands.
No ligands found


#### rem_hydrogen
Detects and remove hydrogen atoms. 
MD setup can be done with the original H atoms, however to prevent from non standard labelling, remove them is safer.  
To remove use rem_hydrogen('yes')


In [43]:
st_c.rem_hydrogen()

Running rem_hydrogen.
No residues with Hydrogen atoms found


#### water
Detects water molecules and allows to remove them
Crystallographic water molecules may be relevant for keeping the structure, however in most cases only some of them are required. These can be later added using other methods (titration) or manually.

To remove water molecules use water('yes')


In [44]:
st_c.water()

Running water.
Detected 80 Water molecules


In [45]:
st_c.water("yes")

Running water. Options: yes
Detected 80 Water molecules
Canonical sequence for model 0:
Removed 80 Water molecules


#### amide
Amide terminal atoms in Asn ang Gln residues can be labelled incorrectly.  
amide suggests possible fixes by checking the sourrounding environent.

To fix use amide ('All | None | residue_list')

Note that the inversion of amide atoms may trigger additional contacts. 

In [46]:
st_c.amide()

Running amide.
Found 6 unusual contact(s) involving amide atoms
 VAL A59.O    ASN A63.OD1     2.784 A
 ALA A80.O    GLN A101.OE1    2.931 A
 GLN A81.OE1  ASN A103.OD1    2.859 A
 ASN A134.ND2 ASN A137.N      2.987 A
 GLU A150.O   ASN A154.OD1    2.871 A
 ARG E357.NH1 ASN E394.ND2    2.964 A


Fix all amide residues and recheck

In [47]:
st_c.amide('all')

Running amide. Options: all
Found 6 unusual contact(s) involving amide atoms
 VAL A59.O    ASN A63.OD1     2.784 A
 ALA A80.O    GLN A101.OE1    2.931 A
 GLN A81.OE1  ASN A103.OD1    2.859 A
 ASN A134.ND2 ASN A137.N      2.987 A
 GLU A150.O   ASN A154.OD1    2.871 A
 ARG E357.NH1 ASN E394.ND2    2.964 A
Amide residues fixed all (7)
Rechecking
Found 4 unusual contact(s) involving amide atoms
 GLN A81.NE2  ASN A103.ND2    2.859 A
 ASN A103.OD1 ASN A194.OD1    2.485 A
 ARG E357.NH1 ASN E394.ND2    3.058 A
 ASN E394.OD1 GLU E516.OE2    2.870 A


Comparing both checks it becomes clear that GLN A42, GLN E498, ASN A103, and ASN A194 are getting new contacts as thay have both changed, ASN E394 is worse as it has now two contacts

In [48]:
st_c.amide('A42,A103')

Running amide. Options: A42,A103
Found 4 unusual contact(s) involving amide atoms
 GLN A81.NE2  ASN A103.ND2    2.859 A
 ASN A103.OD1 ASN A194.OD1    2.485 A
 ARG E357.NH1 ASN E394.ND2    3.058 A
 ASN E394.OD1 GLU E516.OE2    2.870 A
Amide residues fixed A42,A103 (1)
Rechecking
Found 2 unusual contact(s) involving amide atoms
 ARG E357.NH1 ASN E394.ND2    3.058 A
 ASN E394.OD1 GLU E516.OE2    2.870 A


In [49]:
st_c.amide('E394')

Running amide. Options: E394
Found 2 unusual contact(s) involving amide atoms
 ARG E357.NH1 ASN E394.ND2    3.058 A
 ASN E394.OD1 GLU E516.OE2    2.870 A
Amide residues fixed E394 (1)
Rechecking
Found 1 unusual contact(s) involving amide atoms
 ARG E357.NH1 ASN E394.ND2    2.964 A


#### chiral
Side chains of Thr and Ile are chiral, incorrect atom labelling lead to the wrong chirality.  
To fix use chiral('All | None | residue_list')

In [50]:
st_c.chiral()

Running chiral.
Found no residues with incorrect side-chain chirality


#### Backbone
Detects and fixes several problems with the backbone
use any of 
--fix_atoms All|None|Residue List 
--fix_chain All|None|Break list
--add_caps All|None|Terms|Breaks|Residue list
--no_recheck
--no_check_clashes


In [51]:
st_c.backbone()

Running backbone.
Found 2 Residues with missing backbone atoms
 ASP A615   OXT
 GLY E526   OXT
No backbone breaks
No unexpected backbone links


In [52]:
st_c.backbone('--fix_atoms All --fix_chain none --add_caps none')

Running backbone. Options: --fix_atoms All --fix_chain none --add_caps none
Found 2 Residues with missing backbone atoms
 ASP A615   OXT
 GLY E526   OXT
No backbone breaks
No unexpected backbone links
Capping terminal ends
True terminal residues: A19,A615,E333,E526
No caps added
Fixing missing backbone atoms
Adding missing backbone atoms
ASP A615
  Adding new atom OXT
GLY E526
  Adding new atom OXT
Fixed 2 backbone atom(s)
Checking for steric clashes
No severe clashes detected
No apolar clashes detected
No polar_acceptor clashes detected
No polar_donor clashes detected
No positive clashes detected
No negative clashes detected


#### fixside
Detects and re-built missing protein side chains.   
To fix use fixside('All | None | residue_list')

In [53]:
st_c.fixside()

Running fixside.
Found no residues with missing or unknown side chain atoms


#### getss
Detects possible -S-S- bonds based on distance criteria.
Proper simulation requires those bonds to be correctly set. Use All|None|residueList to mark them

In [54]:
st_c.getss()

Running getss.
Detected 7 Possible SS Bonds
 CYS A133.SG  CYS A141.SG     4.215
 CYS A344.SG  CYS A361.SG     4.188
 CYS A530.SG  CYS A542.SG     4.089
 CYS E336.SG  CYS E361.SG     4.142
 CYS E379.SG  CYS E432.SG     4.212
 CYS E391.SG  CYS E525.SG     4.166
 CYS E480.SG  CYS E488.SG     4.253


In [55]:
st_c.getss('all')

Running getss. Options: all
Detected 7 Possible SS Bonds
 CYS A133.SG  CYS A141.SG     4.215
 CYS A344.SG  CYS A361.SG     4.188
 CYS A530.SG  CYS A542.SG     4.089
 CYS E336.SG  CYS E361.SG     4.142
 CYS E379.SG  CYS E432.SG     4.212
 CYS E391.SG  CYS E525.SG     4.166
 CYS E480.SG  CYS E488.SG     4.253
Canonical sequence for model 0:


#### Add_hydrogens
 Add Hydrogen Atoms. Auto: std changes at pH 7.0. His->Hie. pH: set pH value
    list: Explicit list as [*:]HisXXHid, Interactive[_his]: Prompts for all selectable residues
    Fixes missing side chain atoms unless --no_fix_side is set
    Existing hydrogen atoms are removed before adding new ones unless --keep_h set.

In [56]:
st_c.add_hydrogen()

Running add_hydrogen.
Found 226 Residues requiring selection on adding H atoms


In [57]:
st_c.add_hydrogen('auto')

Running add_hydrogen. Options: auto
Found 226 Residues requiring selection on adding H atoms
Running fixside. Options: --fix all
Found no residues with missing or unknown side chain atoms
Selection: auto
Replacing HIS A34 by HIE
Replacing HIS A195 by HIE
Replacing HIS A228 by HIE
Replacing HIS A239 by HIE
Replacing HIS A241 by HIE
Replacing HIS A265 by HIE
Replacing HIS A345 by HIE
Replacing HIS A373 by HIE
Replacing HIS A374 by HIE
Replacing HIS A378 by HIE
Replacing HIS A401 by HIE
Replacing HIS A417 by HIE
Replacing HIS A493 by HIE
Replacing HIS A505 by HIE
Replacing HIS A535 by HIE
Replacing HIS A540 by HIE
Replacing HIS E519 by HIE


#### clashes
Detects steric clashes based on distance criteria.  
Contacts are classified in: 
* Severe: Too close atoms, usually indicating superimposed structures or badly modelled regions. Should be fixed.
* Apolar: Vdw colissions.Usually fixed during the simulation.
* Polar and ionic. Usually indicate wrong side chain conformations. Usually fixed during the simulation


In [58]:
st_c.clashes()

Running clashes.
No severe clashes detected
3 Steric apolar clashes detected
 HIE A34.CD2  TYR E453.OH     2.884 A
 THR A229.CG2 TYR A516.OH     2.863 A
 HIE A417.CE1 SER A545.OG     2.889 A
4 Steric polar_acceptor clashes detected
 MET A152.O   GLY A268.O      3.062 A
 VAL A212.O   ASP A216.OD1    2.940 A
 LEU A333.O   MET A360.O      2.943 A
 GLY E485.O   CYX E488.O      3.023 A
1 Steric polar_donor clashes detected
 ARG E357.NH1 ASN E394.ND2    2.964 A
No positive clashes detected
No negative clashes detected


Complete check in a single method

In [59]:
st_c.checkall()

Running models.
Detected 1 Model(s)
Found Single model
Running chains.
Detected 2 Chain(s)
 A: Protein
 E: Protein
Running inscodes.
Found no residues with insertion codes
Running altloc.
Detected no residues with alternative location labels
Running rem_hydrogen.
Detected 791 Residues containing H atoms
Running add_hydrogen.
Found 209 Residues requiring selection on adding H atoms
Running water.
No water molecules found
Running metals.
No metal ions found
Running ligands.
No ligands found
Running getss.
Detected 7 Possible SS Bonds
 CYX A133.SG  CYX A141.SG     4.215
 CYX A344.SG  CYX A361.SG     4.188
 CYX A530.SG  CYX A542.SG     4.089
 CYX E336.SG  CYX E361.SG     4.142
 CYX E379.SG  CYX E432.SG     4.212
 CYX E391.SG  CYX E525.SG     4.166
 CYX E480.SG  CYX E488.SG     4.253
Running amide.
Found 1 unusual contact(s) involving amide atoms
 ARG E357.NH1 ASN E394.ND2    2.964 A
Running chiral.
Found no residues with incorrect side-chain chirality
Running chiral_bck.
Found no residues 

In [60]:
st_c._save_structure(args['output_structure_path'])

'/home/gelpi/DEVEL/BioPhysics/wdir/6m0j_fixed.pdb'

In [61]:
st_c.rem_hydrogen('yes')

Running rem_hydrogen. Options: yes
Detected 791 Residues containing H atoms
Hydrogen atoms removed from 791 residues


In [63]:
#st_c.add_hydrogen('--add_charges --add_mode auto')
#Alternative way calling through command line
import os
os.system('check_structure -i ' + args['output_structure_path'] + ' -o ' + args['output_structure_path_charges'] + ' add_hydrogen --add_charges --add_mode auto')

=                   BioBB structure checking utility v3.13.4                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-23            =

Structure /home/gelpi/DEVEL/BioPhysics/wdir/6m0j_fixed.pdb loaded
 PDB id:  
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  791
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 791 (total 6102 H atoms)
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  12510
Running add_hydrogen. Options: --add_charges --add_mode auto
Found 209 Residues requiring selection on adding H atoms


usage: add_hydrogen [-h] [--add_mode ADD_MODE] [--pH PH] [--list LIST]
                    [--no_fix_side] [--keep_h] [--add_charges ADD_CHARGES]
add_hydrogen: error: argument --add_charges: expected one argument


512

In [ ]:
#st_c._save_structure(args['output_structure_path_charges'])

In [ ]:
#st_c.revert_changes()